# Similarity metrics for registration

Here, we will learn about similarity metrics.

Specifically, we will compare two popular metrics: 

* [Mean squared error (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error#Predictor)
* [Normalized mutual information (NMI)](https://en.wikipedia.org/wiki/Mutual_information#Normalized_variants)

In this example, we'll first observes that MSE fails, and then understand why it fails. Then observe that using NMI succeeds, and understand why that is the case.

First, let's import libraries and make some useful functions:

In [ ]:
import numpy as np
import SimpleITK as sitk

import itkwidgets
from itkwidgets import view

%matplotlib inline
import matplotlib.pyplot as plt

def compare_overlay(moving, fixed):
    farr = sitk.GetArrayFromImage(fixed).astype(np.float32)
    rarr = sitk.GetArrayFromImage(moving).astype(np.float32)

    plt.figure()
    plt.imshow(farr, 'gray', interpolation='none', alpha=0.8)
    plt.imshow(rarr, 'winter', interpolation='none', alpha=0.5)
    
def run_elastix(fixed_image, moving_image, params):
    """
    Runs elastix with the given fixed and moving images,
    and elastix parameters.
    
    Returns the ElastixImageFilter, from which the results
    can be obtained.
    """
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

Read the example images and visualize them in three different ways:

In [ ]:
movingImage = sitk.ReadImage('../sampleImages/ct_2d_moving.tif')
fixedImage  = sitk.ReadImage('../sampleImages/mri_2d.tif')

First we'll simply visualize the images side-by-side, with `itkwidgets` as we did in the `translation_initialization` notebook.

The CT image (moving) is on the left, and the MR image (fixed) is on the right. Notice:

* Features of the brain are visible in the MR but not the CT
* The skull is bright in the CT but dark in the MR.

In [ ]:
itkwidgets.compare(movingImage,fixedImage)

Next we'll use the "checkerboard" visualization that comes with `itkwidgets`.  Try changing the `pattern` parameter to 12, for example.

In [ ]:
itkwidgets.checkerboard(movingImage,fixedImage, pattern=12)

Last, let's overlay the images using the function defined above.

The main point is that the images are not currently well aligned.

In [ ]:
compare_overlay(movingImage, fixedImage)

Let's load a set of elastix parameters from a file.

In [ ]:
params_mse = sitk.ReadParameterFile('../elastixParameters/AffineMSE_2d.txt')
sitk.PrintParameterMap( params_mse )

Notice the line `(Metric "AdvancedMeanSquares")`, indicating that elastix will use mean squared error. The [documentation](https://elastix.lumc.nl/doxygen/classelastix_1_1AdvancedMeanSquaresMetric.html).

Now, run elastix with these parameters, and visualize the results by overlaying the images.

In [ ]:
elastix_mse = run_elastix(fixedImage, movingImage, params_mse)
result_mse = elastix_mse.GetResultImage()

In [ ]:
compare_overlay(result_mse, fixedImage)

In [1]:
import os

In [5]:
# os.listdir('../slides/figures')
os.listdir('../slides/figures')

['tutfigs',
 'MIPlot.R',
 'mi.pdf',
 'transforms',
 'MSEPlot.R',
 'transformationTypes.png',
 'cor.pdf',
 'imgRegFig_v2.pdf',
 'ItkImageGeometry.png',
 'mse.pdf',
 'imgRegFig-3.pdf',
 'cor.png',
 'elastixLogo.png',
 'imgRegFig_v2-3.pdf',
 'mse1.png',
 'imgRegFig_v2-1.pdf',
 'imgRegFig_v2-2.pdf',
 'imgRegFig_v2-4.pdf',
 'emboBiaCourse2023',
 'CorPlot.R',
 'imgRegFig-2.pdf',
 'ImageOriginAndSpacing.pdf',
 'elastixLogo.gif',
 'haesleinhuepf-1088546103866388481_7s.gif',
 'mi.png',
 'imgRegFig-1a.pdf',
 'registrationPipeline.png',
 'mse.png',
 'imgRegFig-1.pdf']

<img src="../slides/figures/mse.png" >

The result is not great. At the bottom of the image, the skull in the CT is overlayed with part of the brain in the MR. But near the top of the image, the skull is too far up.  As well, there is some rotation that was not corrected for.

Let's understand why MSE produced this result.

Now we know why MSE failed. What is the behavior of mutual information.  Let's start by looking at this image ([credit1](https://commons.wikimedia.org/wiki/File:Mutual_Information_Examples.svg),[credit2](https://commons.wikimedia.org/wiki/File:Correlation_examples2.svg)):

<img src="../slides/figures/mse.png" style="width:600px;height:400px;">

Imagine each point in the 

Look at the top two rows, and notice that mean squared error is sensitive both to how much spread there is in the data, and the linear relationship between the data.  Data with a larger spread have a lower mse (the top row has larger or equal MSE.  

Here's the same plot, but now displaying the correlation.

<img src="../slides/figures/cor.png" style="width:600px;height:400px;">

Here's the same plot, but now displaying the mutual information.

<img src="../slides/figures/mi.png" style="width:600px;height:400px;">


Now that we understand, let's try mutual information, by first loading a parameters file:

In [ ]:
params_mi = sitk.ReadParameterFile('../elastixParameters/AffineMI_2d.txt')
sitk.PrintParameterMap( params_mi )

Notice the line `(Metric "AdvancedMattesMutualInformation")`, indicating that elastix will use mutual information. The [documentation](https://elastix.lumc.nl/doxygen/classelastix_1_1AdvancedMattesMutualInformationMetric.html).

Now, run elastix with these parameters, and visualize the results by overlaying the images.

In [ ]:
elastix_mi = run_elastix(fixedImage, movingImage, params_mi)
result_mi = elastix_mi.GetResultImage()

In [ ]:
compare_overlay(result_mi, fixedImage)